In [1]:
# import argparse


# def parse():
#     parser = argparse.ArgumentParser("tabular_training")

#     # Data
#     parser.add_argument(
#         "--data_path", type=str, default=".", help="path to your data directory"
#     )

#     ############################ Your Code Here ############################
#     # You can add more arguments like the one above in this space

#     ########################################################################

#     return parser.parse_args()

In [4]:
import pandas as pd
import numpy as np
 
import warnings
warnings.filterwarnings("ignore")



def load_data(x_path):
    # Your code here 
    return pd.read_csv(x_path)

#We arent using this 
# def split_data(x, y, split=0.8):
#     # Your code here
#     return train_x, train_y, test_x, test_y


def preprocess_x(df_x, df_y):
    # Your code here
    if(df_y.empty):

      full_df = df_x
      patientunitstayid = []
      num_rows = []
      offset = []
      height = [] 
      weight = []
      gender = []
      avg_glucose = []
      num_visits = []
      avg_ph = []
      coma = []
      avg_coma = []
      avg_heart = []
      age = []
      o2 = []
      resp_rate = []
      bp_mean =[]

      for val in full_df['patientunitstayid'].unique(): 
          temp = full_df[full_df['patientunitstayid'] == val]
          count = len(temp)
          offset.append(temp['offset'].max()) 
          patientunitstayid.append(val)
          num_rows.append(count) 
          max_weight = temp['admissionweight'].max()
          weight.append(max_weight)
          try:
            age_val = full_df[(full_df['patientunitstayid'] == val) & (full_df['age'].notnull())]['age'].iloc[0]
            if age_val == '> 89':
              age_val = 95
              
            age.append(int(age_val))
          except IndexError:
            age.append(None)
          try:
            height_value = full_df[(full_df['patientunitstayid'] == val) & (full_df['admissionheight'].notnull())]['admissionheight'].iloc[0] 
            height.append(height_value)
          except IndexError:
            height.append(None) 
          try:
            gender.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['gender'].notnull())]['gender'].iloc[0])
          except:
            gender.append(None)

          gcs_df = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'GCS Total')] 
          if len(gcs_df) > 0:
              coma.append(len(gcs_df)) 
              gcs_df['nursingchartvalue'] = pd.to_numeric(gcs_df['nursingchartvalue'], errors='coerce')
              gcs_df = gcs_df.dropna(subset=['nursingchartvalue'])
              avg_coma.append(gcs_df['nursingchartvalue'].mean())
          else:
              coma.append(0) 
              avg_coma.append(20)

          heart_rate_df = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Heart Rate')] 
          if len(heart_rate_df) > 0: 
              heart_rate_df['nursingchartvalue'] = pd.to_numeric(heart_rate_df['nursingchartvalue'], errors='coerce')
              heart_rate_df = heart_rate_df.dropna(subset=['nursingchartvalue'])
              avg_heart.append(heart_rate_df['nursingchartvalue'].mean())
          else: 
              avg_heart.append(None)
          
          avg_ph.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['labname'] == 'pH')]['labresult'].mean())
          avg_glucose.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['labname'] == 'glucose')]['labresult'].mean())
          num_visits.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['unitvisitnumber'].notnull())]['unitvisitnumber'].max())
          ##
          ##resp rate
          rr = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Respiratory Rate')] 
          if len(rr) > 0: 
              rr['nursingchartvalue'] = pd.to_numeric(rr['nursingchartvalue'], errors='coerce')
              rr = rr.dropna(subset=['nursingchartvalue'])
              resp_rate.append(rr['nursingchartvalue'].mean())
          else: 
              resp_rate.append(None)
          ##
          ##o2
          o = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'O2 Saturation')] 
          if len(rr) > 0: 
              o['nursingchartvalue'] = pd.to_numeric(o['nursingchartvalue'], errors='coerce')
              o = o.dropna(subset=['nursingchartvalue'])
              o2.append(o['nursingchartvalue'].mean())
          else: 
              o2.append(None)   
          ##bp
          bdf = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Non-Invasive BP Mean')] 
          if len(rr) > 0: 
              bdf['nursingchartvalue'] = pd.to_numeric(o['nursingchartvalue'], errors='coerce')
              bdf = bdf.dropna(subset=['nursingchartvalue'])
              bp_mean.append(bdf['nursingchartvalue'].mean())
          else: 
              bp_mean.append(None)      

      
      new_df = pd.DataFrame({'patientunitstayid': patientunitstayid,'age': age,'gender': gender, 'height':height,'weight':weight,'num_rows': num_rows, 'offset': offset, 'avg_ph':avg_ph,'avg_glucose':avg_glucose, 'num_visits':num_visits,'num_coma_test': coma,'coma_score_avg':avg_coma, 'heart_rate_avg': avg_heart,'resp_rate_avg':resp_rate, 'o2_saturation_avg': o2,'bp_avg':bp_mean})
  
      new_df['height'] = new_df['height'].fillna(new_df['height'].median())
      new_df['weight'] = new_df['weight'].fillna(new_df['weight'].median()) 
      new_df['avg_glucose'] = new_df['avg_glucose'].fillna(new_df['avg_glucose'].median())
      new_df['num_visits'] = new_df['num_visits'].fillna(new_df['num_visits'].median())
      new_df['avg_ph'] = new_df['avg_ph'].fillna(new_df['avg_ph'].median())
      new_df['heart_rate_avg'] = new_df['heart_rate_avg'].fillna(new_df['heart_rate_avg'].median())
      new_df['age'] = new_df['age'].fillna(new_df['age'].median())
      new_df['coma_score_avg'].fillna(20, inplace=True)
      new_df['gender'].fillna(20, inplace=True)
      gender_map = {'Male': 1, 'Female': 0}
      new_df['gender'] = new_df['gender'].map(gender_map)
      new_df['resp_rate_avg'] = new_df['resp_rate_avg'].fillna(new_df['resp_rate_avg'].median())
      new_df['o2_saturation_avg'] = new_df['o2_saturation_avg'].fillna(new_df['o2_saturation_avg'].median())
      new_df['bp_avg'] = new_df['bp_avg'].fillna(new_df['bp_avg'].median())
      return new_df
    else:  
      df = df_x
      df_y = df_y
      full_df = pd.merge(df, df_y, on='patientunitstayid')
      full_df.drop('Unnamed: 0_y', axis = 1,inplace=True) 
      full_df.drop('Unnamed: 0_x', axis = 1,inplace=True)
      patientunitstayid = []
      num_rows = []
      offset = []
      height = [] 
      weight = []
      gender = []
      discharge_status = []
      avg_glucose = []
      avg_ph = []
      num_visits = []
      coma = []
      avg_coma = []
      avg_heart = []
      age = []
      o2 = []
      resp_rate = []
      bp_mean =[]

      for val in full_df['patientunitstayid'].unique(): 
          
          temp = full_df[full_df['patientunitstayid'] == val]
          discharge_status.append(temp['hospitaldischargestatus'].iloc[0])
          count = len(temp)
          offset.append(temp['offset'].max()) 
          patientunitstayid.append(val)
          num_rows.append(count) 
          max_weight = temp['admissionweight'].max()
          weight.append(max_weight)
          try:
            age_val = full_df[(full_df['patientunitstayid'] == val) & (full_df['age'].notnull())]['age'].iloc[0]
            if age_val == '> 89':
              age_val = 95
              
            age.append(int(age_val) )
          except IndexError:
            age.append(None)
          try:
            height_value = full_df[(full_df['patientunitstayid'] == val) & (full_df['admissionheight'].notnull())]['admissionheight'].iloc[0]
          
            height.append(height_value)
          except IndexError:
            height.append(None) 
          try:
            gender.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['gender'].notnull())]['gender'].iloc[0])
          except:
            gender.append(None) 
          avg_ph.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['labname'] == 'pH')]['labresult'].mean())
          avg_glucose.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['labname'] == 'glucose')]['labresult'].mean())
          num_visits.append(full_df[(full_df['patientunitstayid'] == val) & (full_df['unitvisitnumber'].notnull())]['unitvisitnumber'].max())
        
          gcs_df = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'GCS Total')] 
          if len(gcs_df) > 0:
              coma.append(len(gcs_df)) 
              gcs_df['nursingchartvalue'] = pd.to_numeric(gcs_df['nursingchartvalue'], errors='coerce')
              gcs_df = gcs_df.dropna(subset=['nursingchartvalue'])
              avg_coma.append(gcs_df['nursingchartvalue'].mean())
          else:
              coma.append(0) 
              avg_coma.append(20)

          heart_rate_df = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Heart Rate')] 
          if len(heart_rate_df) > 0: 
              heart_rate_df['nursingchartvalue'] = pd.to_numeric(heart_rate_df['nursingchartvalue'], errors='coerce')
              heart_rate_df = heart_rate_df.dropna(subset=['nursingchartvalue'])
              avg_heart.append(heart_rate_df['nursingchartvalue'].mean())
          else: 
              avg_heart.append(None)
          ##
          ##resp rate
          rr = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Respiratory Rate')] 
          if len(rr) > 0: 
              rr['nursingchartvalue'] = pd.to_numeric(rr['nursingchartvalue'], errors='coerce')
              rr = rr.dropna(subset=['nursingchartvalue'])
              resp_rate.append(rr['nursingchartvalue'].mean())
          else: 
              resp_rate.append(None)
          #o2    
          o = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'O2 Saturation')] 
          if len(rr) > 0: 
              o['nursingchartvalue'] = pd.to_numeric(o['nursingchartvalue'], errors='coerce')
              o = o.dropna(subset=['nursingchartvalue'])
              o2.append(o['nursingchartvalue'].mean())
          else: 
              o2.append(None)   
          #bp 
          bdf = full_df[(full_df['patientunitstayid'] == val) & (full_df['nursingchartcelltypevalname'] == 'Non-Invasive BP Mean')] 
          if len(rr) > 0: 
              bdf['nursingchartvalue'] = pd.to_numeric(o['nursingchartvalue'], errors='coerce')
              bdf = bdf.dropna(subset=['nursingchartvalue'])
              bp_mean.append(bdf['nursingchartvalue'].mean())
          else: 
              bp_mean.append(None)      

      new_df = pd.DataFrame({'patientunitstayid': patientunitstayid,'age': age, 'gender': gender, 'height':height,'weight':weight,'num_rows': num_rows, 'offset': offset, 'avg_ph':avg_ph,'avg_glucose':avg_glucose, 'num_visits':num_visits,'num_coma_test': coma,'coma_score_avg':avg_coma, 'heart_rate_avg': avg_heart,'resp_rate_avg':resp_rate, 'o2_saturation_avg': o2,'bp_avg':bp_mean, 'dischargestatus':discharge_status})
  
      new_df['height'] = new_df['height'].fillna(new_df['height'].median())
      new_df['weight'] = new_df['weight'].fillna(new_df['weight'].median()) 
      new_df['avg_glucose'] = new_df['avg_glucose'].fillna(new_df['avg_glucose'].median())
      new_df['avg_ph'] = new_df['avg_ph'].fillna(new_df['avg_ph'].median())
      new_df['num_visits'] = new_df['num_visits'].fillna(new_df['num_visits'].median())
      new_df['heart_rate_avg'] = new_df['heart_rate_avg'].fillna(new_df['heart_rate_avg'].median())
      new_df['age'] = new_df['age'].fillna(new_df['age'].median())
  
      new_df['gender'].fillna(1, inplace=True)
      new_df['coma_score_avg'].fillna(20, inplace=True)
      gender_map = {'Male': 1, 'Female': 0}
      new_df['gender'] = new_df['gender'].map(gender_map)
      new_df['resp_rate_avg'] = new_df['resp_rate_avg'].fillna(new_df['resp_rate_avg'].median())
      new_df['o2_saturation_avg'] = new_df['o2_saturation_avg'].fillna(new_df['o2_saturation_avg'].median())
      new_df['bp_avg'] = new_df['bp_avg'].fillna(new_df['bp_avg'].median())
      return new_df

In [6]:
!pip install xgboost
import xgboost as xgb

class Model():
    def __init__(self, **kwargs):
        self.model = xgb.XGBClassifier(**kwargs)

    def fit(self, x_train, y_train):
        self.model.fit(x_train, y_train)

    def predict_proba(self, x):
        return self.model.predict_proba(x)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00:00:010:01


In [7]:
# from project_utils import create_data_for_project

# data = create_data_for_project(".")

import itertools

 
import pandas as pd
from sklearn.metrics import roc_auc_score 


def main():
    #args = parse()

    x = load_data("data/train_x.csv")
    y = load_data("data/train_y.csv") 

    ###### Your Code Here #######
    # Add anything you want here

    ############################ 
    train_df = preprocess_x(x, y)

    ###### Your Code Here #######
    # Add anything you want here
    X = train_df.drop(['patientunitstayid','dischargestatus'], axis=1)
    y = train_df['dischargestatus']

    ############################

    model = Model(booster='gbtree', learning_rate=0.01, max_depth=6, n_estimators=250, reg_alpha=0.5, reg_lambda=0.75, min_child_weight=1, subsample=1, colsample_bytree=1.0, gamma=0, scale_pos_weight=1, max_delta_step=0)
  
    model.fit(X.drop(['gender','num_visits'],axis=1), y)
    x = load_data("data/test_x.csv")

    ###### Your Code Here #######
    # Add anything you want here

    ############################

    processed_x_test = preprocess_x(x, pd.DataFrame())

    prediction_probs = model.predict_proba(processed_x_test.drop(['patientunitstayid','gender','num_visits' ] ,axis=1))[:, 1]

    #### Your Code Here ####
    # Save your results
    df = pd.DataFrame({'patientunitstayid': processed_x_test['patientunitstayid'].astype('int32'), 'hospitaldischargestatus': prediction_probs}) 
    df.to_csv('predictionXGBoost.csv', index=False)
    ########################


if __name__ == "__main__":
    main()